In [2]:
%matplotlib inline
import matplotlib.pyplot as plt 
import pandas

from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit 
from numpy import log, exp, linspace, sqrt, diag, array, nan
from numpy import concatenate  

In [3]:
# define logistic equation 
def f(x, x0, k): 
    return 1/(1+exp(-k*(x-x0)))

empty = array( [ nan ] * 4 ) 

def fit( df ):
    'Utility function to fit a mutant to logistic equation' 
  
    # scale the rates using the mean of top 10% 
    # more robust than scaling to 1 (gives better fits)
    name = df.name
    mean_highest_rates = df[ df.rate > df.rate.quantile(.9) ].rate.mean()  
    df.rate = df.rate / mean_highest_rates
    
    # linear fit gets us sensible starting params for the logistic fit 
    reg = LinearRegression()
    reg.fit( df.temp.reshape(-1, 1), df.rate )
    slope = reg.coef_[0]
    
    my_index = ['tm', 'k', 'err_tm', 'err_k' ]
    # try fitting to logistic eqn using approximate params from linear fit
    try:
        p0 = ( df.temp.mean(), slope )
        popt, pcov = curve_fit( f, df.temp, df.rate, p0=p0 )
        perr = sqrt( diag( pcov ) ) 
        
        # error checking 
        
        my_params = concatenate( [ popt, perr ] ) 
        if 20 < popt[0] < 60 and popt[1] < 0 and perr[0] < 1: 
            # biological assay limits, and make sure k is the right sign
            return pandas.Series( my_params, index=my_index ) 
        else:
            return pandas.Series( empty, index=my_index )
    except Exception as e:
        print name, e 
        return pandas.Series( empty, index=my_index )
    # done with error checking 

In [4]:
df = pandas.read_csv( 'corrected_assay_data.csv' )
print df.mutant.unique()

['A192S' 'BglB' 'C167A' 'C167Q' 'D403A' 'E154D' 'E164A' 'E164G' 'E164R'
 'E177A' 'E177K' 'E177L' 'E222A' 'E222H' 'E222K' 'E222Q' 'E222R' 'E222Y'
 'E353A' 'E406A' 'E406D' 'E426S' 'F415A' 'F415N' 'F72H' 'G355A' 'H101R'
 'H119A' 'H119E' 'H119N' 'H178A' 'H178R' 'H315E' 'H315N' 'H373R' 'H379R'
 'H379T' 'I244E' 'I244N' 'I300N' 'I91E' 'K341A' 'L171A' 'L171R' 'L362M'
 'M261D' 'M323A' 'M323K' 'M358T' 'N163A' 'N163C' 'N163D_2' 'N163D' 'N220A'
 'N220R' 'N220Y' 'N220G' 'N220H' 'N293A' 'N293C' 'N293D' 'N293Q' 'N354A'
 'N404A' 'N404C' 'P329N' 'Q19A' 'Q19C' 'Q19S' 'Q313R_2' 'Q313R' 'R240A'
 'R240D' 'R240E' 'R240K' 'R76A' 'S14A' 'S16A_2' 'S16N' 'S17A' 'S17E' 'S16A'
 'S298E' 'S331A' 'S400A' 'T15A' 'T175R' 'T218A' 'T296A' 'T352A' 'V52G'
 'W120A' 'W120F' 'W120H' 'W325A' 'W325G' 'W325L_2' 'W325R' 'W325C' 'W325H'
 'W325L' 'W34A' 'W399A' 'W399C' 'W399G' 'W399R' 'W399S' 'W407A' 'W407G'
 'W407Q' 'W407R' 'W409Y' 'Y166P' 'Y18A' 'Y294A' 'Y294F_2' 'Y295A' 'Y295G'
 'Y294F' 'M221A']


In [5]:
# apply the function to the data set 
grouped = df.groupby( by='mutant' )
est = grouped.apply( fit )

print 'Tm estimated for {} of {} samples'.format( len( est.dropna() ), len( grouped ) )

/usr/local/bin/anaconda/lib/python2.7/site-packages/scipy/optimize/minpack.py:690: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


Tm estimated for 75 of 120 samples


In [6]:
print est.round( 2 )

            tm     k  err_tm  err_k
mutant                             
A192S    39.24 -1.29    0.29   0.46
BglB     39.93 -1.19    0.09   0.08
C167A    39.84 -1.47    0.54   0.58
C167Q    38.86 -0.65    0.12   0.05
D403A      NaN   NaN     NaN    NaN
E154D    38.83 -0.77    0.32   0.18
E164A      NaN   NaN     NaN    NaN
E164G      NaN   NaN     NaN    NaN
E164R      NaN   NaN     NaN    NaN
E177A    37.54 -0.52    0.24   0.06
E177K    36.73 -0.70    0.25   0.10
E177L    39.60 -0.76    0.42   0.20
E222A    36.86 -0.69    0.12   0.05
E222H    34.91 -0.70    0.15   0.07
E222K    38.97 -0.59    0.44   0.14
E222Q    39.75 -1.09    0.28   0.23
E222R    39.10 -0.86    0.21   0.14
E222Y    37.13 -0.93    0.51   0.32
E353A      NaN   NaN     NaN    NaN
E406A    40.08 -1.71    0.71   0.86
E406D    40.67 -1.12    0.26   0.27
E426S    39.50 -1.49    0.09   0.13
F415A      NaN   NaN     NaN    NaN
F415N      NaN   NaN     NaN    NaN
F72H     38.99 -0.48    0.38   0.08
G355A      NaN   NaN     NaN

In [7]:
# I looked at all the plots 

zero_list = [ 
    'E353A', 'H315E', 'M261D', 'P329N', 
    'Q313R', 'R76A', 'S16A', 'S16N', 
    'W325H', 'W325L', 'W407G', 'Y294F', 
]

for zero in zero_list:
    est.loc[ zero ] = empty  

In [8]:
for index, dat in grouped:
    dat = dat[ ( dat.rate > 0 ) ] 
    mean_highest_rates = dat[ dat.rate > dat.rate.quantile(.9) ].rate.mean()  
    rate = dat.rate / mean_highest_rates
    my_params = est.loc[ index ]

    plt.figure( figsize=(5,4) )
    plt.scatter( dat.temp, rate, color='k', marker='x', label=index )
    if my_params.tm > 1:
        x = linspace( dat.temp.min(), dat.temp.max(), 50 )
        plt.plot( x, f( x, *my_params[0:2] ), color='k', label='est. params', lw=2 )
        plt.plot( x, f( x, 40.24, -1.45 ), color='g', label='native BglB' )
        
        plt.fill_between( 
            x, 
            f( x, my_params[0]-my_params[2], my_params[1] ), 
            f( x, my_params[0]+my_params[2], my_params[1] ), 
            lw=0,
            color='lightblue',
            alpha=0.5,
        )
        
    plt.xlabel( 'T (C)' )
    plt.ylabel( 'Normalized rate')
    plt.xticks( [ 30, 40, 50 ] )
    plt.yticks( [ 0, 0.25, .5, 0.75, 1 ] )
    plt.xlim( 25, 55 ) 
    plt.ylim( -0.5, 1.5 ) 
    plt.legend( loc='lower left' )
    plt.tight_layout()
    plt.savefig( 'plots/%s.pdf' % index, format='pdf' )
    plt.close()

In [9]:
# diagnostics from June 21

# should have Tm but don't 
# S16A, W325L, Y294F

# diagnostics from Jul 26 

# +  F415A might have a Tm if assay data is cleaned up 
# + Y294F has two sets of assay data together (fixed)
# + N163D has two sets of assay data together (fixed)
# + Q313R has two sets of assay data together (fixed)
# +  S16A has two sets of assay data together (fixed)
# + W325L has two sets of assay data together (fixed)

In [10]:
# let's quickly check against the "published" set

pub = pandas.read_csv( '../../experimental_data/thermo_paper_data_set.csv', index_col=0 )

my = est.join( pub, rsuffix='_pub' ) 
my['same?'] = my.tm == my.tm_pub 
my[ [ 'tm', 'tm_pub' ] ] 

,tm,tm_pub
mutant,,
A192S,39.239899,39.24
BglB,39.925911,39.93
C167A,39.835084,39.84
C167Q,38.858263,38.86
D403A,NaN,NaN
E154D,38.830607,38.83
E164A,NaN,NaN
E164G,NaN,NaN
E164R,NaN,NaN


In [11]:
my.to_csv( 'my_new_tms.csv' ) 

In [12]:
my.loc[ 'M221A' ]

tm                             NaN
k                              NaN
err_tm                         NaN
err_k                          NaN
sequence_pos                   221
expression                       1
tm_pub                         NaN
why_no_tm                      NaN
kcat                           547
km                            6.25
kcatkm                       87554
ki                             NaN
eki                            NaN
percent_err_kcat               2.7
err_kcat                        15
percent_err_km                 9.6
err_km                         0.6
err_kcatkm                    8701
in_plos_paper                    1
gel_number                       4
k_pub                          NaN
err_tm_pub                     NaN
err_k_pub                      NaN
Siena gel levels               0.5
Alex gel levels     should have Tm
same?                        False
Name: M221A, dtype: object